In [9]:
from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
import os


In [10]:
from bsm_utils import BSMUtils
from langchain.tools import Tool
import json
from dotenv import load_dotenv

In [11]:
import bsm_utils
print(dir(bsm_utils))

['BSMUtils', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'norm', 'np', 'pd', 'tool']


In [12]:
load_dotenv()

True

In [13]:
bsm_tool = Tool(
    name="BSM_Calculator",
    func=BSMUtils.black_scholes,
    description="Calculates Black-Scholes-Merton option price. Inputs: option_type, S, K, T, r, sigma."
)

csv_reader_tool = Tool(
    name="CSV_Reader",
    func=BSMUtils.read_csv_tool,
    description="Reads a CSV file and returns the first few rows. Inputs: filepath (str), nrows (int, optional)."
)
###Should we convert csv into json?###

llm = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-5-sonnet-20240620", 
)

# Create the agent
memory = MemorySaver()
tools = [bsm_tool, csv_reader_tool]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

inputs = {"messages": [HumanMessage(content="Calculates Black-Scholes-Merton option price. Inputs: Read the first 5 rows from [dummy_options.csv] using the CSV Reader tool.")]}
config = {"configurable": {"thread_id": "thread_1"}}
# Run the agent with a prompt and thread_id
response = agent_executor.invoke(inputs, config=config)

print(response)

{'messages': [HumanMessage(content='Calculates Black-Scholes-Merton option price. Inputs: Read the first 5 rows from [dummy_options.csv] using the CSV Reader tool.', additional_kwargs={}, response_metadata={}, id='f2132f53-3ec4-4072-809d-82bbea2bd045'), AIMessage(content=[{'text': 'Certainly! I\'ll use the CSV_Reader tool to read the first 5 rows from the file "dummy_options.csv" and then use that information to calculate the Black-Scholes-Merton option price using the BSM_Calculator. Let\'s start by reading the CSV file:', 'type': 'text'}, {'id': 'toolu_01KvQ8wLM2w5dwQtEmjDEDns', 'input': {'__arg1': '{"filepath": "dummy_options.csv", "nrows": 5}'}, 'name': 'CSV_Reader', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01B1KxGYrsNNVYt6VxZ7DWwB', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'c